In [1]:
# 1
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.keys import Keys
from time import sleep
from selenium.webdriver.common.by import By
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import pandas as pd
import csv
import re
from nltk.tag import pos_tag
from nltk.help import upenn_tagset
import chromedriver_autoinstaller
import subprocess
import shutil

In [2]:
# 2 --> 같은 부분 다시 모을 꺼면 일단 링크 
def abs_token(year, abst):
    tokenized_abst = sent_tokenize(str(abst))
    
    stop_words = set(stopwords.words('english'))
    tokenized_sent = []
    test = []
    test_lemmatized_words = []
    tokenizer=TreebankWordTokenizer()
    lemmatizer = WordNetLemmatizer()
    
    for sent in tokenized_abst:
        tokenized_sent = tokenizer.tokenize(sent)
        
        for word in tokenized_sent:
            word = word.lower()
            c = re.sub('[^a-zA-Z]','',word)
            if c != '':
                test.append(c)
        
    #tagged_text = [noun for noun in pos_tag(test) if noun[1].startswith('N')]
    tagged_text = pos_tag(test)

    for word in tagged_text:
        if word[1].startswith('V'):
            w_pos = 'v'
        elif word[1].startswith('N'):
            w_pos = 'n'
        elif word[1].startswith('J'):
            w_pos = 'a'
        elif word[1].startswith('R'):
            w_pos = 'r'
        else:
            continue
        
        processed_word = lemmatizer.lemmatize(word[0],w_pos)
        
        if len(processed_word) > 2 and processed_word not in stop_words:
            test_lemmatized_words.append(processed_word)
        
        
    #print(test_lemmatized_words)
    
    year = str(year)
    with open(year+'.csv','a', newline = '', encoding='utf-8') as f:
        wr = csv.writer(f)
        #for word in test_lemmatized_words:
        #    wr.writerow([word])
        wr.writerow(test_lemmatized_words)

In [3]:
# 3
year_list = ['2011', '2012', '2013','2014','2015','2016','2017','2018','2019','2020']

In [4]:
# 4
# abstract 저장한 url를 퍼블리셔 별로 저장하여 중복 없게 확인용
url_file = open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/scrappedlist/springerlink.csv','a+', newline = '', encoding='utf-8')
url_writer = csv.writer(url_file)
url_reader = csv.reader(url_file)

In [5]:
# 5
driver = webdriver.Chrome('C:/Users/happy/chromedriver/chromedriver')

## 암묵적으로 웹 자원 로드를 위해 3초까지 기다려 준다.
driver.implicitly_wait(3)

In [60]:
# 6
## 시작할 페이지에 접근한다.
one = 'airline'
two = 'repair'
three = 'link.springer.com'
url = 'https://scholar.google.co.kr/scholar?start=0&q=%22'+one+'%22+AND+%22'+two+'%22+AND+site:'+three+'&hl=ko&as_sdt=0,5&as_ylo=2011&as_yhi=2020'
driver.get(url)

In [67]:
# 7 (다른 검색어 긁을 때는 여기서 부터 시작)
# 이미 abstract 가져온 링크 혹시나 있나 확인
scrapped_url = []
url_file.seek(0)

for row in url_reader:
    #print(row)
    scrapped_url += row
    
print(len(scrapped_url))

1923


In [63]:
# 8 --> 같은 페이지 링크 다시 모을 때마다 실행 해줘야 함
url_list = [] # url 리스트 초기화 용

In [64]:
# 9 --> 같은 부분 다시 모을 꺼면 일단 링크 
tmp_list = []
next_condit = True
page_num = 0

while next_condit == True:
    page_num += 1
    
    sleep(10)         # 혹시 구글이 막을까 대기
    req = driver.page_source
    soup = BeautifulSoup(req,'html.parser')
    tmp_list = soup.find_all('h3',{'class':'gs_rt'}) # 링크 찾기

    for tmp in tmp_list:
        try:
            tmp_url = tmp.find('a').get('href')

            if tmp_url not in url_list and '[PDF]' not in tmp.text and 'books' not in str(tmp_url) and tmp_url not in scrapped_url and tmp_url not in url_list:
                url_list.append(tmp_url)

        except Exception as e:
            print(tmp.text, e)
            continue
    
    #다음페이지
    # 주의 페이지 넘기는 표시가 google로 되어 있어야 함. 안그러면 인식 못해요ㅠ
    #if(driver.find_element_by_link_text("다음")):
    #    next_condit = False
    
    #if(soup.select('#gs_n > center > table > tbody > tr > td:nth-child(8) > span')):
    #    next_condit = False
    
    #elif (driver.find_element_by_link_text("다음")):
    #    next_page = driver.find_element_by_link_text("다음")
    #    next_page.click() 
        
    try:
        next_page = driver.find_element_by_link_text("다음")
        next_page.click()
        
    except:
        next_condit = False
        
print(len(url_list))

784


In [65]:
# 10 --> 같은 부분 다시 모을 꺼면 일단 링크 
n = 0
cnt = 0

abstract = ''

for url in url_list:
    
    for url_check in url_reader:
        if url_check == url:
            continue
    
    n += 1
    print(n)
    
    try:
        p_link = url
        driver.get(p_link)
    
    except Exception as e:
        continue
        
    sleep(1)
    req1 = driver.page_source
    soup1 = BeautifulSoup(req1,'html.parser')


    try:
        # abstractSection abstractInFull 구역이 있는지 확인하고 있으면
        # Taylor & Francis / inderscienceonline.com / journals.sagepub.com
        if(soup1.find('div',{'class':'abstractSection abstractInFull'})):
            abstract = soup1.find('div',{'class':'abstractSection abstractInFull'}).text
            # taylor 연도
            if(soup1.find('div',{'class':'itemPageRangeHistory'})):
                year = soup1.find('div',{'class':'itemPageRangeHistory'}).text
                for y in year_list:
                    if y in str(year):
                        year = y
                        break
                    else:
                        continue
            
            # inderscience 연도 --> 이거 페이지 형식 바뀜 다시 확인해 봐야함
            elif(soup1.find('div',{'class':'epub-section'})):
                tmp_soup = soup1.find('div',{'class':'epub-section'})
                year = tmp_soup.find('span',{'class':'epub-section__date'}).text
                for y in year_list:
                    if y in str(year):
                        year = y
                        break
                    else:
                        continue
            #sage            
            elif(soup1.find('span',{'class':'publicationContentEpubDate dates'})):
                year = soup1.find('span',{'class','publicationContentEpubDate dates'}).text
                for y in year_list:
                    if y in str(year):
                        year = y
                        break
                    else:
                        continue
            
            driver.back()         

        # ieeexplore.ieee.org
        elif(soup1.find('div',{'class':'abstract-text row'})):
            tmp_soup = soup1.find('div',{'class':'abstract-text row'})
            abstract_tmp = soup1.find('div',{'class':'abstract-text row'}).text
            abstract = abstract_tmp.replace('Abstract:','')

            year = soup1.find('div',{'class':'u-pb-1 stats-document-abstract-publishedIn'})
            for y in year_list:
                if y in str(year):
                    year = y
                    break
                else:
                    continue
            
            driver.back()

        elif(soup1.select('#abstract')):
            tmp_soup = soup1.select_one('#abstract')
            
            # emerald.com
            if(tmp_soup.select('#abstract > div:nth-child(2) > section > p')):
                abstract_tmp = tmp_soup.find_all('p')
                for a in abstract_tmp:
                    if(a is not []):
                        abstract += a.text
                #mainContent > div:nth-child(3) > div > div > header > div > div:nth-child(1) > p:nth-child(4) > span.intent_journal_publication_date        
                year = soup1.select_one('#mainContent > div:nth-child(3) > div > div > header > div > div:nth-child(1) > p:nth-child(3) > span.intent_journal_publication_date')
                for y in year_list:
                    if y in str(year):
                        year = y
                        break
                    else:
                        continue
                driver.back()
                
            #sustainability
            elif(soup1.select('#abstract > div:nth-child(3) > div.target-item.custom-accordion-for-small-screen-content > div.art-abstract.in-tab.hypothesis_container')):
                abstract = soup1.select_one('#abstract > div:nth-child(3) > div.target-item.custom-accordion-for-small-screen-content > div.art-abstract.in-tab.hypothesis_container').text

                year = soup1.select_one('#abstract > div.content__container.content__container__combined-for-large.content__container__combined-for-large__first > div.bib-identity').text
                for y in year_list:
                    if y in str(year):
                        year = y
                        break
                    else:
                        continue
                        
                driver.back()
                
            #journal.oscm-forum.org
            elif(soup1.select_one('#abstract').text):
                abstract_tmp = soup1.select_one('#abstract').text
                for a in abstract_tmp:
                    if(a is not []):
                        abstract += a

                year = soup1.select_one('#top > div:nth-child(2) > div > div:nth-child(1) > ol > li:nth-child(2)').text

                for y in year_list:
                    if y in str(year):
                        year = y
                        break
                    else:
                        continue
                
                driver.back()
                
            else:
                driver.back()
        
        #science direct
        elif(soup1.select('#abstracts')):
            tmp_soup = soup1.select_one('#abstracts')
            abstract = tmp_soup.find('p')
            year = soup1.select_one('#publication > div.publication-volume')
            for y in year_list:
                if y in str(year):
                    year = y
                    break
                else:
                    continue
            driver.back()
        
        #JSTOR
        elif(soup1.select('#content > div.row > div > div:nth-child(2) > div > div > div.turnaway-summary-preview > p')):
            tmp_soup = soup1.select_one('#content > div.row > div > div:nth-child(2) > div > div > div.turnaway-summary-preview')
            abstract = tmp_soup.find('p').text
            print(abstract)

            year = soup1.select_one('#content > div.row > div > div.row.turnaway-padding.turnaway-content-summary-container > div.columns.mbl.small-12.large-8.xlarge-4.medium-push-12.large-push-12.xlarge-push-8 > div > div.turn-away-content__article-summary-info > div.turn-away-content__summary.row > div:nth-child(1) > div:nth-child(2) > a').text
            for y in year_list:
                if y in str(year):
                    year = y
                    break
                else:
                    continue
                driver.back()
        
        #SpringerLink
        elif(soup1.find('section',{'class':'Abstract'})):
            tmp_soup = soup1.find('section',{'class':'Abstract'})  #Abs1-content
            abstract_tmp = tmp_soup.find_all('p')

            for a in abstract_tmp:
                abstract = abstract + a.text

            year = soup1.find('div',{'class':'article-dates'}).text

            for y in year_list:
                if y in str(year):
                    year = y
                    break
                else:
                    continue
            
        else:
            print(p_link, '필터 안됨')
            driver.back()
            continue
            
        # 대상 연도 아니면 저장말고 넘어가기
        if year not in year_list:
            print(p_link, '대상연도 아님')
            continue

        else:
            # 전처리 함수로 이동
            abs_token(year, abstract)
            cnt += 1
            
            # abstract 받은 링크 저장 
            url_writer.writerow([url])
            #print("링크 저장")
        
        
        
    except Exception as e:
        print(p_link, '오류명:',e)
        driver.back()


1
https://link.springer.com/article/10.1007/s00170-016-8520-9 필터 안됨
2
https://link.springer.com/chapter/10.1007/978-3-030-68476-1_15 대상연도 아님
3
https://link.springer.com/content/pdf/10.1007/s00411-013-0460-2.pdf 필터 안됨
4
5
6
https://link.springer.com/article/10.1007/s12652-018-0997-7 필터 안됨
7
https://link.springer.com/content/pdf/10.1007/978-3-662-43373-7.pdf 필터 안됨
8
https://link.springer.com/chapter/10.1007/978-3-319-17055-8_111 오류명: 'NoneType' object has no attribute 'text'
9
https://link.springer.com/chapter/10.1007/978-1-4419-7750-2_9 대상연도 아님
10
https://link.springer.com/article/10.1007/s12063-016-0109-9 필터 안됨
11
12
13
https://link.springer.com/article/10.1111/jere.12243 필터 안됨
14
https://link.springer.com/chapter/10.1007/978-1-4302-3678-8_5 오류명: 'NoneType' object has no attribute 'text'
15
https://link.springer.com/chapter/10.1007/978-3-642-32775-9_17 오류명: 'NoneType' object has no attribute 'text'
16
https://link.springer.com/article/10.1007/s10288-010-0145-5 필터 안됨
17
18
https://link.

131
https://link.springer.com/chapter/10.1007/978-3-642-39955-8_7 오류명: 'NoneType' object has no attribute 'text'
132
133
https://link.springer.com/content/pdf/10.1007/s11628-012-0181-7.pdf 필터 안됨
134
135
136
https://link.springer.com/chapter/10.1057/9781137472854_2 오류명: 'NoneType' object has no attribute 'text'
137
138
139
https://link.springer.com/article/10.1140/epjb/e2016-70618-0 필터 안됨
140
141
142
https://link.springer.com/chapter/10.1057/9781137316271_2 오류명: 'NoneType' object has no attribute 'text'
143
https://link.springer.com/article/10.1007/s00170-010-2767-3 필터 안됨
144
https://link.springer.com/chapter/10.1007/978-3-642-40361-3_52 오류명: 'NoneType' object has no attribute 'text'
145
146
https://link.springer.com/article/10.3103/S1068798X20070126 필터 안됨
147
https://link.springer.com/chapter/10.1007/978-1-4302-4618-3_8 오류명: 'NoneType' object has no attribute 'text'
148
https://link.springer.com/article/10.1057/s41299-017-0022-7 필터 안됨
149
https://link.springer.com/chapter/10.1007/978-3

286
https://link.springer.com/chapter/10.1007/978-3-319-48764-9_191 오류명: 'NoneType' object has no attribute 'text'
287
288
289
https://link.springer.com/article/10.1007/s10479-018-2933-9 필터 안됨
290
https://link.springer.com/chapter/10.1007/978-3-642-30820-8_28 오류명: 'NoneType' object has no attribute 'text'
291
292
293
https://link.springer.com/content/pdf/10.1007/978-3-319-52459-7.pdf 필터 안됨
294
https://link.springer.com/content/pdf/10.1007/s00502-014-0221-1.pdf 필터 안됨
295
https://link.springer.com/article/10.1007/s11846-016-0208-x 필터 안됨
296
https://link.springer.com/article/10.1057/s41283-019-00052-0 필터 안됨
297
https://link.springer.com/article/10.1007/s00264-014-2444-5 필터 안됨
298
https://link.springer.com/chapter/10.1057/9781137386199_4 오류명: 'NoneType' object has no attribute 'text'
299
https://link.springer.com/article/10.1007/s00192-014-2575-7 필터 안됨
300
301
302
303
https://link.springer.com/article/10.1007/s12532-019-00172-4 필터 안됨
304
https://link.springer.com/article/10.1007/s40279-015

436
437
https://link.springer.com/article/10.1007/s00268-013-2320-2 필터 안됨
438
https://link.springer.com/content/pdf/10.1007/s00383-011-2907-3.pdf 필터 안됨
439
https://link.springer.com/content/pdf/10.1007/978-94-009-3237-1.pdf 필터 안됨
440
https://link.springer.com/article/10.1007/s11047-016-9609-7 필터 안됨
441
https://link.springer.com/chapter/10.1007/978-1-349-68398-7_178 오류명: 'NoneType' object has no attribute 'text'
442
443
https://link.springer.com/chapter/10.1057/9781137393999_2 오류명: 'NoneType' object has no attribute 'text'
444
https://link.springer.com/article/10.1134/S0362119715070026 필터 안됨
445
https://link.springer.com/article/10.1134/S1075700712030033 필터 안됨
446
447
448
449
https://link.springer.com/content/pdf/10.1007/978-1-349-57823-8_321.pdf 필터 안됨
450
451
https://link.springer.com/chapter/10.1007/978-3-662-47407-5_6 오류명: 'NoneType' object has no attribute 'text'
452
https://link.springer.com/article/10.1007/s10845-016-1241-y 필터 안됨
453
https://link.springer.com/article/10.1007/s1177

598
599
600
601
https://link.springer.com/article/10.1057/s41272-020-00245-3 필터 안됨
602
https://link.springer.com/article/10.1007/s11661-011-0703-2 필터 안됨
603
604
https://link.springer.com/content/pdf/10.1007/978-981-10-3911-9.pdf 필터 안됨
605
606
https://link.springer.com/content/pdf/10.1007/978-1-349-57823-8_281.pdf 필터 안됨
607
https://link.springer.com/content/pdf/10.1007/s11067-014-9278-y.pdf 필터 안됨
608
https://link.springer.com/article/10.1007/s00170-018-2205-5 필터 안됨
609
610
611
612
https://link.springer.com/article/10.1007/s40171-015-0107-z 필터 안됨
613
https://link.springer.com/article/10.1007/s00167-013-2592-y 필터 안됨
614
615
https://link.springer.com/chapter/10.1007/978-1-137-57165-6_2 오류명: 'NoneType' object has no attribute 'text'
616
617
618
https://link.springer.com/chapter/10.1007/978-3-642-24541-1_32 오류명: 'NoneType' object has no attribute 'text'
619
https://link.springer.com/article/10.1007/s13412-015-0342-9 필터 안됨
620
621
https://link.springer.com/chapter/10.1007/978-1-4842-0385-9_1 

In [66]:
print('총 탐색',n,'개 중', cnt, '개 수집')

총 탐색 784 개 중 330 개 수집


In [68]:
url_file.close()

In [69]:
url_list.clear()

In [ ]:
------------------ 이 아래는 테스트 용 실행 안하셔도 됩니다.-----------------------

In [8]:
url = 'https://link.springer.com/chapter/10.1007/978-3-642-19689-8_4'

driver.get(url)

sleep(10)
req1 = driver.page_source
soup1 = BeautifulSoup(req1,'html.parser')

In [9]:
# 링크 접속
try:
    abstract = ''
    # abstractSection abstractInFull 구역이 있는지 확인하고 있으면
    # Taylor & Francis / inderscienceonline.com / journals.sagepub.com
    if(soup1.find('div',{'class':'abstractSection abstractInFull'})):
        abstract = soup1.find('div',{'class':'abstractSection abstractInFull'}).text
        # taylor 연도
        if(soup1.find('div',{'class':'itemPageRangeHistory'})):
            year = soup1.find('div',{'class':'itemPageRangeHistory'}).text
            for y in year_list:
                if y in str(year):
                    year = y
                    break
                else:
                    continue
        
        # inderscience 연도    
        elif(soup1.find('div',{'class':'epub-section'})):
                tmp_soup = soup1.find('div',{'class':'epub-section'})
                year = tmp_soup.find('span',{'class':'epub-section__date'}).text
                for y in year_list:
                    if y in str(year):
                        year = y
                        break
                    else:
                        continue

        #sage            
        elif(soup1.find('span',{'class':'publicationContentEpubDate dates'})):
            year = soup1.find('span',{'class','publicationContentEpubDate dates'}).text
            for y in year_list:
                if y in str(year):
                    year = y
                    break
                else:
                    continue

    # ieeexplore.ieee.org
    elif(soup1.find('div',{'class':'abstract-text row'})):
        tmp_soup = soup1.find('div',{'class':'abstract-text row'})
        abstract_tmp = soup1.find('div',{'class':'abstract-text row'}).text
        abstract = abstract_tmp.replace('Abstract:','')
        
        year = soup1.find('div',{'class':'u-pb-1 stats-document-abstract-publishedIn'})
        
        for y in year_list:
            if y in str(year):
                year = y
                break
            else:
                continue
        
    elif(soup1.select('#abstract')):
        tmp_soup = soup1.select_one('#abstract')

        # emerald.com
        if(tmp_soup.select('#abstract > div:nth-child(2) > section > p')):
            abstract_tmp = tmp_soup.find_all('p')
            for a in abstract_tmp:
                if(a is not []):
                    abstract += a.text
                
            year = soup1.select_one('#mainContent > div:nth-child(3) > div > div > header > div > div:nth-child(1) > p:nth-child(4) > span.intent_journal_publication_date')
            for y in year_list:
                if y in str(year):
                    year = y
                    break
                else:
                    continue
                    
        #sustainability
        elif(soup1.select('#abstract > div:nth-child(3) > div.target-item.custom-accordion-for-small-screen-content > div.art-abstract.in-tab.hypothesis_container')):
            abstract = soup1.select_one('#abstract > div:nth-child(3) > div.target-item.custom-accordion-for-small-screen-content > div.art-abstract.in-tab.hypothesis_container').text           
            
            year = soup1.select_one('#abstract > div.content__container.content__container__combined-for-large.content__container__combined-for-large__first > div.bib-identity').text
            for y in year_list:
                if y in str(year):
                    year = y
                    break
                else:
                    continue

        #journal.oscm-forum.org
        elif(soup1.select_one('#abstract')):
            abstract_tmp = soup1.select_one('#abstract').text
            for a in abstract_tmp:
                if(a is not []):
                    abstract += a
                    
            year = soup1.select_one('#top > div:nth-child(2) > div > div:nth-child(1) > ol > li:nth-child(2)').text
            
            for y in year_list:
                if y in str(year):
                    year = y
                    break
                else:
                    continue
        else:
            print('\n',p_link,'\n')


    #science direct
    elif(soup1.select('#abstracts')):
        tmp_soup = soup1.select_one('#abstracts')
        abstract = tmp_soup.find('p')
        year = soup1.select_one('#publication > div.publication-volume')
        for y in year_list:
            if y in str(year):
                year = y
                break
            else:
                continue

    #JSTOR
    elif(soup1.select('#content > div.row > div > div:nth-child(2) > div > div > div.turnaway-summary-preview > p')):
        tmp_soup = soup1.select_one('#content > div.row > div > div:nth-child(2) > div > div > div.turnaway-summary-preview')
        abstract = tmp_soup.find('p').text
        print(abstract)
        
        year = soup1.select_one('#content > div.row > div > div.row.turnaway-padding.turnaway-content-summary-container > div.columns.mbl.small-12.large-8.xlarge-4.medium-push-12.large-push-12.xlarge-push-8 > div > div.turn-away-content__article-summary-info > div.turn-away-content__summary.row > div:nth-child(1) > div:nth-child(2) > a').text
        for y in year_list:
            if y in str(year):
                year = y
                break
            else:
                continue

    #SpringerLink
    elif(soup1.find('section',{'class':'Abstract'})):
        tmp_soup = soup1.find('section',{'class':'Abstract'})
        abstract_tmp = tmp_soup.find_all('p')
        
        for a in abstract_tmp:
            abstract = abstract + a.text
        
        year = soup1.find('div',{'class':'article-dates'}).text
        
        for y in year_list:
            if y in str(year):
                year = y
                break
            else:
                continue

    else:
        print('\n',url, '필터 안됨\n')
        
    if year not in year_list:
        print('대상 연도 아님')
    
    else:
        print(year)
        print(abstract)
        abs_token(year, abstract)
    

except Exception as e:
    print('\n',url, '오류명:',e,'\n')

2011
The aviation maintenance, repair and overhaul (MRO) industry has a long history of developing and applying product service systems and similar service packages. Aircraft operator value the idea of shifting part of the operating risk to service providers by having them guarantee aircraft availability. The revenue model for such services is often based on operated flight hours and thereby closely related to the revenue model of an aircraft operator. Nevertheless, true total care packages for aircraft have not yet prevailed on the market since aircraft operators prefer modular packages that offer them higher cost transparency and more flexibility in deciding what activities to outsource.Consumable and Expendable Material in the aviation industry is used during all MRO activities especially during airframe maintenance and overhaul. The material scope ranges from standard hardware like nuts, bolts and rivets to liquid consumables such as oils, greases and hydraulic fluids to electrical

In [35]:
def abs_token(year, abst):
    tokenized_abst = sent_tokenize(str(abst))
    
    stop_words = set(stopwords.words('english'))
    tokenized_sent = []
    test = []
    test_lemmatized_words = []
    tokenizer=TreebankWordTokenizer()
    lemmatizer = WordNetLemmatizer()
    
    for sent in tokenized_abst:
        tokenized_sent = tokenizer.tokenize(sent)
        
        for word in tokenized_sent:
            word = word.lower()
            c = re.sub('[^a-zA-Z]','',word)
            if c != '':
                test.append(c)
        
    #tagged_text = [noun for noun in pos_tag(test) if noun[1].startswith('N')]
    tagged_text = pos_tag(test)

    for word in tagged_text:
        if word[1].startswith('V'):
            w_pos = 'v'
        elif word[1].startswith('N'):
            w_pos = 'n'
        elif word[1].startswith('J'):
            w_pos = 'a'
        elif word[1].startswith('R'):
            w_pos = 'r'
        else:
            continue
        
        processed_word = lemmatizer.lemmatize(word[0],w_pos)
        
        if len(processed_word) > 2 and processed_word not in stop_words:
            test_lemmatized_words.append(processed_word)
        
        
    #print(test_lemmatized_words)
    
    year = str(year)
    with open(year+'.csv','a', newline = '', encoding='utf-8') as f:
        wr = csv.writer(f)
        #for word in test_lemmatized_words:
        #    wr.writerow([word])
        wr.writerow(test_lemmatized_words)

In [ ]:
page_num = 15 #크롤링 할 페이지 수 입력 // 20페이지 씩 끊어서 담기!!

tmp_list = []

for i in range(page_num):
    sleep(10)         # 혹시 구글이 막을까 대기
    req = driver.page_source
    soup = BeautifulSoup(req,'html.parser')
    tmp_list = soup.find_all('h3',{'class':'gs_rt'}) # 링크 찾기

    for tmp in tmp_list:
        try:
            tmp_url = tmp.find('a').get('href')

            if tmp_url not in url_list and '[PDF]' not in tmp.text and 'books' not in str(tmp_url) and tmp_url not in scrapped_url and tmp_url not in url_list:
                url_list.append(tmp_url)

        except Exception as e:
            print(tmp.text, e)
            continue
    
    #다음페이지
    # 주의 페이지 넘기는 표시가 google로 되어 있어야 함. 안그러면 인식 못해요ㅠ
    if(i != page_num-1):
        next_page = driver.find_element_by_link_text("다음")
        next_page.click()
        
print(len(url_list))

In [35]:
# 5
subprocess.Popen(r'C:/Program Files/Google/Chrome/Application/chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"') # 디버거 크롬 구동

option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
driver.implicitly_wait(10)